# 参数管理

## 访问参数，用于调试、诊断和可视化。
## 参数初始化。
## 在不同模型组件间共享参数。


In [1]:
import tensorflow as tf

X = tf.random.uniform((2, 4))

In [2]:
net = tf.keras.models.Sequential([
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8, activation=None),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(1, activation=None),
])
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.7668282],
       [0.7198111]], dtype=float32)>

## **访问指定block/layer**

我们从已有模型中访问参数。
当通过`Sequential`类定义模型时，
我们可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。
如下所示，我们可以检查第二个全连接层的参数。

## Sequential生成一个layers (List),  layers[2]对应第二个Dense 

In [3]:
print(net.layers[2].weights)

[<tf.Variable 'dense_1/kernel:0' shape=(8, 1) dtype=float32, numpy=
array([[-0.6598364 ],
       [ 0.56728673],
       [-0.2959383 ],
       [-0.00194317],
       [-0.5172061 ],
       [ 0.08133674],
       [ 0.533018  ],
       [-0.6375403 ]], dtype=float32)>, <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


输出的结果告诉我们一些重要的事情：
首先，这个全连接层包含两个参数，分别是该层的权重和偏置。
两者都存储为单精度浮点数（float32）。
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

## **访问指定block/层 的参数**

注意，每个参数都表示为参数类的一个实例。
要对参数执行任何操作，首先我们需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值。


In [4]:
print("type: ",type(net.layers[2].weights[1]))
print("bias: ",net.layers[2].weights[1])
print("data: ",tf.convert_to_tensor(net.layers[2].weights[1]))

type:  <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
bias:  <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
data:  tf.Tensor([0.], shape=(1,), dtype=float32)


### [**一次性访问所有参数**]

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要递归整个树来提取每个子块的参数。
下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。


In [5]:
print(net.layers[0].weights)
print(net.get_weights())

[<tf.Variable 'dense/kernel:0' shape=(4, 8) dtype=float32, numpy=
array([[-0.09296751,  0.6679663 , -0.26512727,  0.11755329, -0.25970736,
         0.5232691 ,  0.02066123,  0.11952746],
       [ 0.35446423,  0.34212703, -0.51548696, -0.0973683 , -0.38872206,
         0.32615966,  0.4805705 ,  0.07160157],
       [-0.6638046 ,  0.62671834, -0.32958803,  0.45271677, -0.4523997 ,
         0.25104195, -0.3976754 ,  0.0191974 ],
       [-0.28820673, -0.04724914, -0.5159568 , -0.32207525,  0.42924017,
        -0.43402064,  0.25565994, -0.5466225 ]], dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(8,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]
[array([[-0.09296751,  0.6679663 , -0.26512727,  0.11755329, -0.25970736,
         0.5232691 ,  0.02066123,  0.11952746],
       [ 0.35446423,  0.34212703, -0.51548696, -0.0973683 , -0.38872206,
         0.32615966,  0.4805705 ,  0.07160157],
       [-0.6638046 ,  0.62671834, -0.32958803,  0.45271677, -0.4523997 ,

这为我们提供了另一种访问网络参数的方式，如下所示。


In [6]:
net.get_weights()[2]

array([[-0.6598364 ],
       [ 0.56728673],
       [-0.2959383 ],
       [-0.00194317],
       [-0.5172061 ],
       [ 0.08133674],
       [ 0.533018  ],
       [-0.6375403 ]], dtype=float32)

## **从嵌套块收集参数**

让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。


In [7]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(8, activation=tf.nn.relu),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add(block1(name=f'block {i}'))
        #net.add(block1() ) must has a name
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00671816],
       [0.00836153]], dtype=float32)>

[**设计了网络后，我们看看它是如何工作的。**]
# 输出整个网络结构
## 由于一开始有两个Dense() 所以这里的dense_10应该是dense_8
## 结果可以和pytorch版对上 pytorch里bias默认的初始方式不是0

In [8]:
print(rgnet.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    multiple                  304       
_________________________________________________________________
dense_10 (Dense)             multiple                  5         
Total params: 309
Trainable params: 309
Non-trainable params: 0
_________________________________________________________________
None


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。


In [9]:
rgnet.layers[0].layers[1].layers[0].weights[1]

<tf.Variable 'dense_4/bias:0' shape=(8,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

  
  
# Parameter Initialization !!!!!!!!!

默认情况下，Keras会根据一个范围均匀地初始化权重矩阵，
这个范围是根据输入和输出维度计算出的。
偏置参数设置为0。
TensorFlow在根模块和`keras.initializers`模块中提供了各种初始化方法。


# Initialization API

让我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
且将偏置参数设置为0。


## use normal distribution

In [10]:
net = tf.keras.models.Sequential([
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        #################################
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
        #################################
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-1.8330323e-02,  7.7645108e-03, -1.1331505e-03, -1.1228327e-02],
        [ 6.6171340e-03,  1.5305574e-03, -6.4847386e-03, -1.0005854e-02],
        [ 9.8848912e-05, -4.0707539e-04,  1.0280007e-02,  3.6858730e-03],
        [ 1.0790724e-02,  6.5617948e-03,  6.5857540e-03,  1.1158202e-02]],
       dtype=float32)>,
 <tf.Variable 'dense_11/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

我们还可以将所有参数初始化为给定的常数，比如初始化为1。


In [11]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_13/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

我们还可以[**对某些块应用不同的初始化方法**]。
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层，
然后将第三个神经网络层初始化为常量值42。


In [12]:
net = tf.keras.models.Sequential([
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(1)),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_16/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>


# **自定义初始化**

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


在这里，我们定义了一个`Initializer`的子类，
并实现了`__call__`函数。
该函数返回给定形状和数据类型的所需张量。


In [13]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[0].weights[:2])

[<tf.Variable 'dense_17/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.       ,  0.       ,  0.       ,  0.       ],
       [-0.       , -7.187631 ,  6.187065 , -0.       ],
       [ 7.4667625, -0.       , -0.       , -9.085226 ],
       [-9.590261 , -7.6465607,  7.628973 , -0.       ]], dtype=float32)>, <tf.Variable 'dense_17/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]


# 我们始终直接设置参数。
# .assign()

In [14]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_18/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.        ],
       [ 1.6122347 ],
       [ 1.4670157 ],
       [ 0.39684832]], dtype=float32)>

# 参数共享

有时我们希望在多个层间共享参数：
我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。


In [15]:
# tf.keras的表现有点不同。它会自动删除重复层
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# 检查参数是否不同
print(len(net.layers) == 3)

True
